In [ ]:
import os
import shutil
os.chdir('C:/Users/USERNAME/')

In [ ]:
import os
import shutil
os.makedirs('EasyMocap/data/boxes', exist_ok=True)
shutil.copy('EasyMocap/yamls/REORIENTED/extri.yml','EasyMocap/data/boxes/')
shutil.copy('EasyMocap/yamls/REORIENTED/intri.yml','EasyMocap/data/boxes/')

In questa parte vedremo come trasformare un file 3d in modo da farlo leggere dal nostro programma python.

La pipeline potrà sembrare strana e complicata, tuttavia è ancora in fase di sviluppo.

Iniziamo.

Le nostre bounding box sono i volumi in cui quando le mani vanno possiamo dire che l'operatore sta lavorando su un componente X.

Per definire questi volumi, è possibile farlo a mano scrivendo a mano un file .yml dove si definiscono le coordinate del vertice (della box) piu vicino e piu lontano dall'origine strutturato come il file NOTEBOOKS/boxes_example.

In alternativa è possibile avvalersi di un software chiamato free cad, in cui è possibile definire i volumi, salvarli in formato STEP, definire mesh e successivamente trasformarli in file .yml attraverso un comando da terminale freecad (è possibile trovare anche questo comando in notebooks NOTEBOOKS/STEP_TO_YML.py).

Nel caso in cui in data non sia presente la cartella boxes, crearla, dopodichè porci all'interno il file boxes.yml appena creato.

Per controllare la struttura corretta è possibile utilizzare la funzione viewboxes....

In [ ]:
!python EasyMocap/apps/hopeitworks/viewboxes.py EasyMocap/data/boxes/boxes.yml

Per proiettare invece le boxes nel nostro sistema reale (immagini dei video), possiamo utilizzare la seguente cella:

In [ ]:
!python EasyMocap/apps/hopeitworks/boxesprojection.py EasyMocap/data --out EasyMocap/data/boxes/ --boxes-yml EasyMocap/data/boxes/boxes.yml --write

Ora ripuliamo la cartella data utilizzata precedentemente per il motioncapture dei keypoints per definire O-X-Y (se popolato) e procediamo con il caricamento dei video in data/videos ( mi raccomando eliminare prima quelli vecchi!)e il rilevamento dei keypoints come visto nel tutorial 3... 

In [ ]:
import shutil
import os

!python EasyMocap/apps/hopeitworks/cleanhandmocap.py
os.remove('EasyMocap/data/extri.yml')  
os.remove('EasyMocap/data/intri.yml')  
shutil.copy('EasyMocap/yamls/REORIENTED/extri.yml','EasyMocap/data/')
shutil.copy('EasyMocap/yamls/REORIENTED/intri.yml','EasyMocap/data/')

Ripetiamo quindi i passaggi gia visti nel tutorial 3: images -> to annots -> to kp3d

In [ ]:
##ESTRAE IMMAGINI DA VIDEO
import time
import os
# Inizio del timer
start_time = time.time()

# ESTRAZIONE IMMAGINI
!python EasyMocap/apps/preprocess/extract_image.py EasyMocap/data

# Fine del timer
end_time = time.time()
elapsed_time = end_time - start_time
# Salva il tempo in un file txt
with open('EasyMocap/data/times.txt', 'a') as f:
    f.write(f't extract_image, {elapsed_time:.2f}\n')
##ESTRAE KEYPOINTS
import time
import os
import json
import shutil
# Inizio del timer
start_time = time.time()

## MANO SINISTRA
!python EasyMocap/apps/preprocess/extract_keypoints.py EasyMocap/data --mode mp-handl
## RINOMINA ANNOTS IN ANNOTSL PER CONVENIENZA DEL SUCCESSIVO ANNOT (DELLA MANO DESTRA)
os.rename('EasyMocap/data/annots', 'EasyMocap/data/annotsl')
## MANO DESTRA
!python EasyMocap/apps/preprocess/extract_keypoints.py EasyMocap/data --mode mp-handr


##INIZIO JOIN DELLE DUE CARTELLE ANNOTS

# Define the path to your main folder
main_folder_path = 'EasyMocap/data/annotsl'

# Get a list of sub-folders within the main folder
subfolders = [f.path for f in os.scandir(main_folder_path)]

# Loop through each sub-folder
for subfolder_path in subfolders:

    # Get a list of JSON files within the sub-folder
    json_files = [f.path for f in os.scandir(subfolder_path)]

    # Loop through the JSON files in the sub-folder
    for json_file_path in json_files:

        # You can now read and process the JSON file
        with open(json_file_path, 'r') as json_file:
            data = json.load(json_file)
            json_file_path_dest = json_file_path.replace("annotsl", "annots")

            # Step 1: Read the existing data from the destination JSON file
            with open(json_file_path_dest, 'r') as json_file:
                existing_data = json.load(json_file)

            # Step 2: Modify or add new data to the existing data
            existing_data["annots"][0]["handl2d"] =  data["annots"][0]["handl2d"]
            existing_data["annots"][0]["bbox_handl2d"] =  data["annots"][0]["bbox_handl2d"]

            # Step 3: Write the updated data back to the JSON file
            with open(json_file_path_dest, 'w') as json_file:
                json.dump(existing_data, json_file, indent=4)

shutil.rmtree('EasyMocap/data/annotsl')
##FINE JOIN DELLE DUE CARTELLE ANNOTS

# Fine del timer
end_time = time.time()
elapsed_time = end_time - start_time
# Salva il tempo in un file txt
with open('EasyMocap/data/times.txt', 'a') as f:
    f.write(f't extract_keypoints, {elapsed_time:.2f}\n')

##in times.txt, SCRIVERE A INIZIO FILE NUMERO FRAME E NUMERO FPS CON QUESTO FORMATO EX. (#FRAMES, 434 /n #FPS, 9)
##TRIANGOLAZIONE K2D->K3D
import time

# Inizio del timer
start_time = time.time()

!python EasyMocap/apps/fit/triangulate1p.py --cfg_data EasyMocap/config/recon/mv1p.yml --opt_data args.path EasyMocap/data args.out EasyMocap/data/output-keypoints3d --cfg_exp EasyMocap/config/recon/triangulator-mv1p-total.yml --opt_exp  args.debug True

# Fine del timer
end_time = time.time()
elapsed_time = end_time - start_time
# Salva il tempo in un file txt
with open('EasyMocap/data/times.txt', 'a') as f:
    f.write(f't triangulate, {elapsed_time:.2f}\n')

**ITERATIVE STEP** 

Ripuliamo quindi la cartella con la seguente cella:

In [ ]:
!python EasyMocap/apps/hopeitworks/cleanboxes.py 

E procediamo con il rilevare se i keypoints e le bounding box entrano in contatto e intersecano tramite la seguente funzione:

In [ ]:
!python EasyMocap/apps/hopeitworks/rilevak3d.py EasyMocap/data/boxes/boxes.yml EasyMocap/data/output-keypoints3d/keypoints3d EasyMocap/data/boxes/results.csv

E con la creazioni delle immagini (questa volta con le box che cambiano di colore):

In [ ]:
!python EasyMocap/apps/hopeitworks/boxesprojection.py EasyMocap/data --out EasyMocap/data/boxes/ --boxes-yml EasyMocap/data/boxes/boxes.yml --results EasyMocap/data/boxes/results.csv --write

La seguente cella ci permetterà di creare un video in cui vedremo i nostri risultati.

In [ ]:
##CREA VIDEO
import cv2
video = cv2.VideoCapture("EasyMocap/data/videos/ORIGINE.mp4");
fps = video.get(cv2.CAP_PROP_FPS)
print("Frames per second using video.get(cv2.CAP_PROP_FPS) : {0}".format(fps))
fps = int(fps)

!python -m easymocap.visualize.ffmpeg_wrapper EasyMocap/data/boxes/boxes/CAM1 --fps $fps
!python -m easymocap.visualize.ffmpeg_wrapper EasyMocap/data/boxes/boxes/CAM2 --fps $fps
!python -m easymocap.visualize.ffmpeg_wrapper EasyMocap/data/boxes/boxes/CAM3 --fps $fps
!python -m easymocap.visualize.ffmpeg_wrapper EasyMocap/data/boxes/boxes/CAM4 --fps $fps

Una volta anallizzati i video, potete scegliere se essere contenti con i risultati, o modificare le geometrie delle bounding box usando il tool presente qui sotto (DA ESEGUIRE IN TERMINALE!) e ripetere il processo ripartendo da l' ITERATIVE STEP..

In [ ]:
!python EasyMocap/apps/hopeitworks/boxestool.py EasyMocap/data/boxes/boxes.yml

AU REVOIR!